In [35]:
import data_processor
from importlib import reload
import pandas as pd
import shutil

In [36]:
reload(data_processor)

<module 'data_processor' from '/Users/hoda_hamdy/Documents/Projects/fleetpy/FleetPy/notebooks/data_processor.py'>

In [37]:
from data_processor import *

In [38]:
scenario = 'example_pool_sc_1'
train_data_dir = f'../studies/GNN_study/results/{scenario}/train/'
save_raw_dir = f'data/{scenario}/raw/'
save_process_dir = f'data/{scenario}/processed/'
sim_duration = 7200  # seconds
sim_step = 60  # seconds

In [39]:
data_processor = DataProcessor(train_data_dir, save_raw_dir, sim_duration, sim_step)
all_data = data_processor.process_data()

100%|██████████| 120/120 [00:00<00:00, 801.59it/s]


In [40]:
shutil.rmtree(f'{save_process_dir}')
os.makedirs(f'{save_process_dir}')
for name in ['req_features', 'veh_features']:
    dfs = []
    for timestep, data in enumerate(all_data):
        df = pd.DataFrame.from_dict(data[name], orient='index')
        df['timestep'] = timestep
        dfs.append(df)
    pd.concat(dfs).to_csv(f'{save_process_dir}/{name}.csv', index_label='id')

In [41]:
node_mapping = {timestep: {rid: ind for ind, rid in enumerate(data['req_features'].keys())} for timestep, data in enumerate(all_data)}

In [42]:
def get_edge_attrs(name, node, nghb, info):
    edge_attrs = {'source': node_mapping[timestep][node] if name == 'rr_graph' else node,
                  'target': node_mapping[timestep][nghb]}
    # TODO use more meaningful naming
    if type(info['travel_cost']) == list:
        for i in range(len(info['travel_cost'])):
            edge_attrs[f'travel_time{i}'] = info['travel_cost'][i]['travel_time']
    else:
        edge_attrs['travel_time'] = info['travel_time']
    edge_attrs['common_nghbs'] = info['common_nghbs']
    edge_attrs['jaccards_coeff'] = info['jaccards_coeff']
    edge_attrs['init_label'] = info.get('init_assign', 0)
    edge_attrs['label'] = info.get('opt_assign', 0)
    return edge_attrs

In [43]:
for name in ['rr_graph', 'vr_graph']:
    dfs = []
    for timestep, data in enumerate(all_data):
        df = [get_edge_attrs(name, node, nghb, info) for node, nghbs in data[name].items() for nghb, info in nghbs.items()]
        df = pd.DataFrame(df)
        df['timestep'] = timestep
        dfs.append(df)
    pd.concat(dfs).to_csv(f'{save_process_dir}/{name}.csv', index=False)